# Backtesting the HMM Model

Signals are identifed by the hmm model and imported into the system. 

    When signal = 1, we enter the mean reversion position.    
    When signal = 0, we enter the momentum position. 


Mean Reversion Strategy:
     
     1.We rank the returns of ETF stocks in the past month
     2.For bottom 3 stocks, we buy     
     3.For top 3 stocks, we sell     
     4.We close the position every 10 days 


Momentum Strategy (using scores): 

     1.We rank the returns of ETF stocks in the past n_i months
     2.Given different n_i, we obtain the ranking score of each stocks
     3.For bottom 3 stocks, we sell     
     4.For top 3 stocks, we buy     
     5.We close the position every month 
    
    
    


In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt

import pandas as pd
from datetime import datetime
import backtrader.analyzers as btanalyzers

In [2]:

class RSI_Oversold(bt.Indicator):
    lines = ('oversold',)
    
    params = (
        ('level', 30),    
    )

    def __init__(self):
        self.lines.oversold = bt.indicators.RSI(self.data) < self.p.level

        
class DummyInd(bt.Indicator):
    lines = ('dummyline',)

    params = (('value', 5),)

    def __init__(self):
        self.lines.dummyline = bt.Max(0.0, self.params.value)

        
# Create a Stratey
class BackTradingStrategy(bt.Strategy):
    params = (
        ('period', 255),
        ("max_days_in_trade", 10)
    )

    def log(self, txt, txt2, txt3, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s, %s, %s' % (dt.isoformat(), txt, txt2, txt3))
    
    
    def log2(self, txt, dt=None):
        ''' Logging function for this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))
        
        
    def read_signals(self):
        signal_data = pd.read_csv("hmm_regime_signals.csv")
        signal_data["datetime"] = signal_data["datetime"].apply(lambda x: datetime.strptime(x,"%m/%d/%Y"))
        signal_data = signal_data.set_index('datetime')
        
        return signal_data
        

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        # How to access individual etf
        self.xlb_close = self.datas[symbols.index("XLB")].close
        self.xle_close = self.datas[symbols.index("XLE")].close
        self.xlf_close = self.datas[symbols.index("XLF")].close
        self.xli_close = self.datas[symbols.index("XLI")].close
        self.xlk_close = self.datas[symbols.index("XLK")].close
        self.xlp_close = self.datas[symbols.index("XLP")].close
        self.xlu_close = self.datas[symbols.index("XLU")].close
        self.xlv_close = self.datas[symbols.index("XLV")].close
        self.xly_close = self.datas[symbols.index("XLY")].close
        
        self.datadate = self.datas[0].datetime.date
        self.days_in_trade = 0
        self.max_days_in_trade_mr = 10
        self.max_days_in_trade_mo = 21
        self.num_mr = 21
        self.num_mo = 21*6
        self.num_mrs = [10, 15, 20, 25]
        self.num_mos = [21*6, 21*8, 21*10, 21*12]
        self.regime_state = 0
        self.mrr = 0.02
        self.mor = 0.05
        self.snum = 3 
        self.cash = 1000
        self.signal_data = self.read_signals()
        # keep this 
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.period)        
        self.symbols = ['XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLU', 'XLV', 'XLY']
        self.regimes = ['momentum', 'mean reversion']
        self.invest = 0
        self.path = []
        
        
    def stock_return(self, n):
        """calculate return in the past n days """        
        r_xlb = (self.xlb_close[0] - self.xlb_close[-n])/self.xlb_close[-n]
        r_xle = (self.xle_close[0] - self.xle_close[-n])/self.xle_close[-n]
        r_xlf = (self.xlf_close[0] - self.xlf_close[-n])/self.xlf_close[-n]
        r_xli = (self.xli_close[0] - self.xli_close[-n])/self.xli_close[-n]
        r_xlk = (self.xlk_close[0] - self.xlk_close[-n])/self.xlk_close[-n]
        r_xlp = (self.xlp_close[0] - self.xlp_close[-n])/self.xlp_close[-n]
        r_xlu = (self.xlu_close[0] - self.xlu_close[-n])/self.xlu_close[-n]
        r_xlv = (self.xlv_close[0] - self.xlv_close[-n])/self.xlv_close[-n]
        r_xly = (self.xly_close[0] - self.xly_close[-n])/self.xly_close[-n]
        r_list = [r_xlb, r_xle, r_xlf, r_xli, r_xlk, r_xlp, r_xlu, r_xlv, r_xly]        
        
        df = pd.DataFrame(r_list, index = self.symbols, columns = ['return'])
        
        return df
    
    
    
    def sort_return(self, n):
        """sort return in the past n days in ascending order 
           return the bottom and top 3 return stocks """
        
        df = self.stock_return(n)
        df = df.sort_values(by=['return'])                
        stock_list = [df.index[0:3].tolist(),df.index[6:].tolist()]
        
        return stock_list
    
    
    def sort_return2(self, nums):
        """sort return in the past n_i days in ascending order 
           For each stock, its rank @ n_i is the score_i
           For each stock, its cumulative score = sum of score_i for all i
           return the bottom and top 3 scored stocks """
        
        n = len(nums)
        snum = len(symbols)
        scores=[0]*snum
        for i in range(n):
            s = []
            df = self.stock_return(nums[i])
            df = df.sort_values(by=['return'])
            sorted_index = df.index.tolist()
            dictionary = self.list_to_dict(sorted_index)
            for symbol in symbols:
                s.append(dictionary[symbol])
            for j in range(snum):
                scores[j] = scores[j] + s[j]
        ddf = pd.DataFrame(scores, index = symbols, columns=['return'])
        ddf = ddf.sort_values(by=['return'])
        stock_list = [ddf.index[0:3].tolist(),ddf.index[snum-3:].tolist()]
        
        return stock_list
        
        
    
    def list_to_dict(self,llist):  
        dct = {} 
        n = len(llist)
        for i in range(n):
            item = llist[i]
            dct[item] = i  
        return dct 

    
    def check_position(self):
        flag = False
        names = []
        for symbol in self.symbols:
            s_size = self.getposition(data=self.datas[symbols.index(symbol)]).size
            if  s_size != 0:
                flag = True
                names.append(symbol)
        return flag, names
    
    
    def shares_to_buy(self,cash, price):
        """calculate the approximate number of shares to buy given cash to spend and stock price"""
        shares = int(cash/price)
        return shares
            
                
    

    def next(self):
        # Simply log the closing price of the series from the reference               
        #print('investment = ', self.invest)
        dt = self.datadate(0).strftime('%Y-%m-%d')
        
        signal = self.signal_data['state'].loc[dt]
        
        position, names = self.check_position()
        
        #self.log('regime = %s' % self.regimes[signal], '      HAS Position at %s' % " ".join(names[:3]), ' and  at  %s' %  " ".join(names[3:]))
        self.log(' %5.4f' % self.invest, '      HAS Position at %s' % " ".join(names[:3]), ' and  at  %s' %  " ".join(names[3:]))

        
        
        # Check if we are in the market
        if not position:
            # if we do not have a position
            
            if signal == 1:
                """mean reversion"""
                
                stock_list = self.sort_return2(self.num_mrs)
                # print('stocklist = ', stock_list)
                
                # self.log('regime = %s' % self.regimes[signal], 'CREATE BUY  for %s' % " ".join(stock_list[0]), ' SELL for %s' %  " ".join(stock_list[0])) 

                # Buy bottom snum stocks
                for i in range(self.snum):
                    price = self.datas[symbols.index(stock_list[0][i])].close[0]
                    shares = self.shares_to_buy(self.cash,price) 
                    long_value = price * shares
                    self.order = self.buy(data = self.datas[symbols.index(stock_list[0][i])], size = shares)
                    
                # Sell top snum stocks    
                for i in range(self.snum):
                    price = self.datas[symbols.index(stock_list[1][i])].close[0]
                    shares = self.shares_to_buy(self.cash, price)
                    self.order = self.sell(data = self.datas[symbols.index(stock_list[1][i])], size = shares)
                    short_value = price*shares
                self.invest = long_value - short_value
                self.path.append([dt, long_value-short_value])

                self.days_in_trade = 1
                
                self.regime_state = 1
                    
                    
         
            
            else:
                """momentum"""  
                
                stock_list = self.sort_return2(self.num_mos)
                #print('stocklist = ', stock_list)
                
                #self.log('regime = %s' % self.regimes[signal], '      CREATE SELL for %s' % " ".join(stock_list[0]), ' BUY  for %s' %  " ".join(stock_list[0]))

                # Sell bottom snum stocks
                for i in range(self.snum):
                    price = self.datas[symbols.index(stock_list[0][i])].close[0]
                    shares = self.shares_to_buy(self.cash,price)
                    short_value = price*shares
                    self.order = self.sell(data = self.datas[symbols.index(stock_list[0][i])], size = shares)
                    
                # Buy top snum stocks    
                for i in range(self.snum):
                    price = self.datas[symbols.index(stock_list[1][i])].close[0]
                    shares = self.shares_to_buy(self.cash,price)
                    long = price*shares
                    long_value = price*shares
                    self.order = self.buy(data = self.datas[symbols.index(stock_list[1][i])], size = shares)
                self.invest = long_value - short_value
                self.path.append([dt, long_value-short_value])
                    
                self.days_in_trade = 1
                
                self.regime_state = 0
                    
            
                
            

        else:
            # if we have a position
            
            self.days_in_trade += 1
            self.path.append([dt, self.invest])
            
            """mean reversion"""
            if (self.regime_state == 1):
            
                # close the position if regime shift or after holding it for max_mr days.
                if signal == 0 or self.days_in_trade == self.max_days_in_trade_mr:
                    num = len(names)
                    for i in range(num):
                        self.close(data=self.datas[symbols.index(names[i])])
                    
                    #self.log('regime = %s' % self.regimes[signal], 'CLOSE Position  %s' % " ".join(names[:3]), ' and for  %s' %  " ".join(names[3:]))
                    self.days_in_trade = 0                    
                    self.regime_state = 0 
                    
            
            """momentum"""
            if (self.regime_state == 0):
                
                # close the position if regime shift or after holding it for max_mo days.
                if signal == 1 or self.days_in_trade == self.max_days_in_trade_mo:
                    num = len(names)
                    for i in range(num):
                        self.close(data=self.datas[symbols.index(names[i])])
                    
                    # self.log('regime = %s' % self.regimes[signal], '      CLOSE Position  %s' % " ".join(names[:3]), ' and for  %s' %  " ".join(names[3:]))
                    self.days_in_trade = 0                        
                    self.regime_state = 1
                 
       


In [3]:


if __name__ == '__main__':
    # Create a cerebro entity
    cerebro = bt.Cerebro()   

    symbols = ['XLB', 'XLE', 'XLF', 'XLI', 'XLK', 'XLP', 'XLU', 'XLV', 'XLY']
    for symbol in symbols:
        datapath = os.path.join('../../../datas/%s.csv' % symbol)

        # Create a Data Feed
        data = bt.feeds.YahooFinanceCSVData(
            dataname=datapath,
            # Do not pass values before this date
            fromdate=datetime(2013, 2, 11),
            # Do not pass values before this date
            todate=datetime(2018, 2, 9),
            # Do not pass values after this date
            reverse=False)

        # Add the Data Feed to Cerebro
        cerebro.adddata(data, name=symbol)


    # Add a strategy
    cerebro.addstrategy(BackTradingStrategy) #, symbols= symbols)
    
    # Set our desired cash start
    cerebro.broker.setcash(100000.0)

    # Write output
    cerebro.addwriter(bt.WriterFile, out='0416-ETFbacktesting-Scored.csv',csv=True)  
    
    
    # Analyzer
    cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='mysharpe')
    cerebro.addanalyzer(btanalyzers.TimeDrawDown, _name='DrawDown')
    cerebro.addanalyzer(btanalyzers.TradeAnalyzer, _name='TA')

    
    # Print out the starting conditions
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Run over everything
    # cerebro.run()
    thestrats = cerebro.run()
    thestrat = thestrats[0]

    # Print out the final result
    print('\nFinal Portfolio Value: %.2f' % cerebro.broker.getvalue())
    
    # Print out sharpe ratio and statistics
    print('Sharpe Ratio:', thestrat.analyzers.mysharpe.get_analysis())
    print('Max Draw Down:', thestrat.analyzers.DrawDown.get_analysis(), "\n")
    
   # for k,v in thestrat.analyzers.TA.get_analysis().items():
   #     print(k, ":", v, "\n")
    

Starting Portfolio Value: 100000.00
2014-02-13,  0.0000,       HAS Position at ,  and  at  
2014-02-14,  6.8800,       HAS Position at XLE XLI XLP,  and  at  XLU XLV XLY
2014-02-18,  6.8800,       HAS Position at XLE XLI XLP,  and  at  XLU XLV XLY
2014-02-19,  6.8800,       HAS Position at XLE XLI XLP,  and  at  XLU XLV XLY
2014-02-20,  6.8800,       HAS Position at XLE XLI XLP,  and  at  XLU XLV XLY
2014-02-21,  6.8800,       HAS Position at XLE XLI XLP,  and  at  XLU XLV XLY
2014-02-24,  6.8800,       HAS Position at XLE XLI XLP,  and  at  XLU XLV XLY
2014-02-25,  6.8800,       HAS Position at XLE XLI XLP,  and  at  XLU XLV XLY
2014-02-26,  6.8800,       HAS Position at XLE XLI XLP,  and  at  XLU XLV XLY
2014-02-27,  6.8800,       HAS Position at XLE XLI XLP,  and  at  XLU XLV XLY
2014-02-28,  6.8800,       HAS Position at XLE XLI XLP,  and  at  XLU XLV XLY
2014-03-03,  6.8800,       HAS Position at XLE XLI XLP,  and  at  XLU XLV XLY
2014-03-04,  6.8800,       HAS Position at XLE XLI

2014-10-31,  13.5900,       HAS Position at XLB XLE XLK,  and  at  XLU XLV XLY
2014-11-03,  13.5900,       HAS Position at XLB XLE XLK,  and  at  XLU XLV XLY
2014-11-04,  13.5900,       HAS Position at XLB XLE XLK,  and  at  XLU XLV XLY
2014-11-05,  13.5900,       HAS Position at XLB XLE XLK,  and  at  XLU XLV XLY
2014-11-06,  13.5900,       HAS Position at XLB XLE XLK,  and  at  XLU XLV XLY
2014-11-07,  13.5900,       HAS Position at XLB XLE XLK,  and  at  XLU XLV XLY
2014-11-10,  13.5900,       HAS Position at XLB XLE XLK,  and  at  XLU XLV XLY
2014-11-11,  13.5900,       HAS Position at XLB XLE XLK,  and  at  XLU XLV XLY
2014-11-12,  13.5900,       HAS Position at ,  and  at  
2014-11-13,  3.3000,       HAS Position at XLB XLE XLK,  and  at  XLU XLV XLY
2014-11-14,  3.3000,       HAS Position at XLB XLE XLK,  and  at  XLU XLV XLY
2014-11-17,  3.3000,       HAS Position at XLB XLE XLK,  and  at  XLU XLV XLY
2014-11-18,  3.3000,       HAS Position at XLB XLE XLK,  and  at  XLU XLV XLY

2015-07-16,  -25.1600,       HAS Position at XLB XLE XLP,  and  at  XLU XLV XLY
2015-07-17,  -25.1600,       HAS Position at XLB XLE XLP,  and  at  XLU XLV XLY
2015-07-20,  -25.1600,       HAS Position at XLB XLE XLP,  and  at  XLU XLV XLY
2015-07-21,  -25.1600,       HAS Position at XLB XLE XLP,  and  at  XLU XLV XLY
2015-07-22,  -25.1600,       HAS Position at XLB XLE XLP,  and  at  XLU XLV XLY
2015-07-23,  -25.1600,       HAS Position at XLB XLE XLP,  and  at  XLU XLV XLY
2015-07-24,  -25.1600,       HAS Position at XLB XLE XLP,  and  at  XLU XLV XLY
2015-07-27,  -25.1600,       HAS Position at XLB XLE XLP,  and  at  XLU XLV XLY
2015-07-28,  -25.1600,       HAS Position at XLB XLE XLP,  and  at  XLU XLV XLY
2015-07-29,  -25.1600,       HAS Position at XLB XLE XLP,  and  at  XLU XLV XLY
2015-07-30,  -25.1600,       HAS Position at XLB XLE XLP,  and  at  XLU XLV XLY
2015-07-31,  -25.1600,       HAS Position at XLB XLE XLP,  and  at  XLU XLV XLY
2015-08-03,  -25.1600,       HAS Positio

2016-01-21,  31.9100,       HAS Position at XLB XLE XLF,  and  at  XLP XLU XLV
2016-01-22,  31.9100,       HAS Position at XLB XLE XLF,  and  at  XLP XLU XLV
2016-01-25,  31.9100,       HAS Position at XLB XLE XLF,  and  at  XLP XLU XLV
2016-01-26,  31.9100,       HAS Position at XLB XLE XLF,  and  at  XLP XLU XLV
2016-01-27,  31.9100,       HAS Position at XLB XLE XLF,  and  at  XLP XLU XLV
2016-01-28,  31.9100,       HAS Position at XLB XLE XLF,  and  at  XLP XLU XLV
2016-01-29,  31.9100,       HAS Position at XLB XLE XLF,  and  at  XLP XLU XLV
2016-02-01,  31.9100,       HAS Position at XLB XLE XLF,  and  at  XLP XLU XLV
2016-02-02,  31.9100,       HAS Position at XLB XLE XLF,  and  at  XLP XLU XLV
2016-02-03,  31.9100,       HAS Position at XLB XLE XLF,  and  at  XLP XLU XLV
2016-02-04,  31.9100,       HAS Position at XLB XLE XLF,  and  at  XLP XLU XLV
2016-02-05,  31.9100,       HAS Position at XLB XLE XLF,  and  at  XLP XLU XLV
2016-02-08,  31.9100,       HAS Position at XLB XLE 

2016-09-02,  -6.5700,       HAS Position at XLB XLF XLI,  and  at  XLU XLV XLY
2016-09-06,  -6.5700,       HAS Position at XLB XLF XLI,  and  at  XLU XLV XLY
2016-09-07,  -6.5700,       HAS Position at XLB XLF XLI,  and  at  XLU XLV XLY
2016-09-08,  -6.5700,       HAS Position at XLB XLF XLI,  and  at  XLU XLV XLY
2016-09-09,  -6.5700,       HAS Position at XLB XLF XLI,  and  at  XLU XLV XLY
2016-09-12,  -6.5700,       HAS Position at XLB XLF XLI,  and  at  XLU XLV XLY
2016-09-13,  -6.5700,       HAS Position at XLB XLF XLI,  and  at  XLU XLV XLY
2016-09-14,  -6.5700,       HAS Position at XLB XLF XLI,  and  at  XLU XLV XLY
2016-09-15,  -6.5700,       HAS Position at XLB XLF XLI,  and  at  XLU XLV XLY
2016-09-16,  -6.5700,       HAS Position at XLB XLF XLI,  and  at  XLU XLV XLY
2016-09-19,  -6.5700,       HAS Position at XLB XLF XLI,  and  at  XLU XLV XLY
2016-09-20,  -6.5700,       HAS Position at ,  and  at  
2016-09-21,  13.8900,       HAS Position at XLF XLK XLP,  and  at  XLU XLV

2017-05-09,  14.2900,       HAS Position at XLE XLF XLI,  and  at  XLK XLP XLV
2017-05-10,  14.2900,       HAS Position at XLE XLF XLI,  and  at  XLK XLP XLV
2017-05-11,  14.2900,       HAS Position at XLE XLF XLI,  and  at  XLK XLP XLV
2017-05-12,  14.2900,       HAS Position at XLE XLF XLI,  and  at  XLK XLP XLV
2017-05-15,  14.2900,       HAS Position at XLE XLF XLI,  and  at  XLK XLP XLV
2017-05-16,  14.2900,       HAS Position at XLE XLF XLI,  and  at  XLK XLP XLV
2017-05-17,  14.2900,       HAS Position at XLE XLF XLI,  and  at  XLK XLP XLV
2017-05-18,  14.2900,       HAS Position at XLE XLF XLI,  and  at  XLK XLP XLV
2017-05-19,  14.2900,       HAS Position at XLE XLF XLI,  and  at  XLK XLP XLV
2017-05-22,  14.2900,       HAS Position at ,  and  at  
2017-05-23,  10.2600,       HAS Position at XLE XLF XLI,  and  at  XLK XLP XLV
2017-05-24,  10.2600,       HAS Position at XLE XLF XLI,  and  at  XLK XLP XLV
2017-05-25,  10.2600,       HAS Position at XLE XLF XLI,  and  at  XLK XLP

2018-01-16,  19.8900,       HAS Position at XLB XLE XLI,  and  at  XLK XLP XLU
2018-01-17,  19.8900,       HAS Position at XLB XLE XLI,  and  at  XLK XLP XLU
2018-01-18,  19.8900,       HAS Position at XLB XLE XLI,  and  at  XLK XLP XLU
2018-01-19,  19.8900,       HAS Position at XLB XLE XLI,  and  at  XLK XLP XLU
2018-01-22,  19.8900,       HAS Position at XLB XLE XLI,  and  at  XLK XLP XLU
2018-01-23,  19.8900,       HAS Position at XLB XLE XLI,  and  at  XLK XLP XLU
2018-01-24,  19.8900,       HAS Position at XLB XLE XLI,  and  at  XLK XLP XLU
2018-01-25,  19.8900,       HAS Position at XLB XLE XLI,  and  at  XLK XLP XLU
2018-01-26,  19.8900,       HAS Position at XLB XLE XLI,  and  at  XLK XLP XLU
2018-01-29,  19.8900,       HAS Position at XLB XLE XLI,  and  at  XLK XLP XLU
2018-01-30,  19.8900,       HAS Position at XLB XLE XLI,  and  at  XLK XLP XLU
2018-01-31,  19.8900,       HAS Position at XLB XLE XLI,  and  at  XLK XLP XLU
2018-02-01,  19.8900,       HAS Position at XLB XLE 

In [4]:
df2 = pd.DataFrame(BackTradingStrategy.path)
print(df2)

AttributeError: type object 'BackTradingStrategy' has no attribute 'path'

In [ ]:
# calculate pnl using excel data
import pandas as pd
import matplotlib.pyplot as plt

dtime = pd.read_csv('datetime.csv')
data = pd.read_csv('0410-ETFbacktesting-Scored.csv').fillna(value = 0)
data = data[:1260]

df = data[data.columns[-9:]].copy()
df.columns = symbols
df['datetime'] = dtime[['datetime']].copy()

n = len(df)
df['cumulative_pnl'] = df['XLB']

cpnl = 0.
count_plus = 0
count_minus = 0

for t in range(n):
    pnl = 0.
    for i in range(len(symbols)):
        profit = df[symbols[i]].iloc[t]
        # print('profit = ', profit)
        pnl = pnl + profit
        if (profit > 0.):
            count_plus +=1
        elif (profit < 0.):
            count_minus +=1
         
    cpnl = cpnl + pnl
    # print('cpnl = ', cpnl)
    df['cumulative_pnl'].iloc[t] = cpnl
    


    

In [ ]:
plus_percent = 1.0*count_plus/(count_plus*1.0+count_minus*1.0) 
print('percentage of trading that make positive profits = ', plus_percent)

In [ ]:
# convert string to datetime
df["datetime"] = df["datetime"].apply(lambda x: datetime.strptime(x,"%m/%d/%Y"))

# plot PNL figures 
plt.figure(figsize=(16, 4))
plt.plot(df['datetime'], df['cumulative_pnl'] ,'.') 
plt.xlabel("Time")
plt.ylabel("cumulative pnl")
plt.title("Daily PNL")
plt.show()

In [ ]:
df.to_csv(r'hmm_regime_wealth_Scored_v1.csv',header=list(df),index=list(df.index),sep=",",mode="a")

In [ ]:
data = pd.read_csv('0410-ETFbacktesting-hmm-pnl.csv')
df = data[:1260]
# convert string to datetime
df["datetime"] = df["datetime"].apply(lambda x: datetime.strptime(x,"%m/%d/%Y"))

# plot figures for pnl of all versions
plt.figure(figsize=(16, 4))
plt.plot(df['datetime'], df['pnl_scored_v1'], label = 'pnl_scored_v1 SR = -5.989')
plt.plot(df['datetime'], df['pnl_scored_v2'], label = 'pnl_scored_v2 SR = -6.616')
plt.plot(df['datetime'], df['pnl_6m'], label = 'pnl_06m SR = -5.930')
plt.plot(df['datetime'], df['pnl_8m'], label = 'pnl_08m SR = -6.478')
plt.plot(df['datetime'], df['pnl_10m'], label = 'pnl_10m SR = -5.239')
plt.plot(df['datetime'], df['pnl_12m'], label = 'pnl_12m SR = -4.872')
plt.legend(loc='best')
plt.show()